In [3]:
from pymongo import MongoClient

# Подключение к локальному серверу MongoDB (по умолчанию порт 27017)
client = MongoClient('mongodb://localhost:27017/')

In [13]:
# Получение списка баз
client.list_database_names()

['admin', 'config', 'local']

In [14]:
# Выбор базы данных (замените 'mydatabase' на нужное имя)
# Если базы еще не существует, то она будет создана
db = client['mydatabase']

Схема и валидация

In [23]:
from jsonschema import validate

qa_schema = {
    "type": "object",
    "properties": {
        "_id": {},
        "question": {"type": "string"},
        "answer": {"type": "string"}
    },
    "required": ["question", "answer"],
    "additionalProperties": False
}

data = {
    "question": "В чем разница между tuple и list?",
    "answer": "Tuple относится к неизменяемым типам данных, list к изменяемым &startA"
}

validate(instance=data, schema=qa_schema)  # Не выбросит ошибку, если валидно

Создание коллекции и добавление данных

In [25]:
collection_name = "python"


if collection_name in db.list_collection_names():
    db.drop_collection(collection_name) 

db.create_collection(collection_name, validator={
    "$jsonSchema": qa_schema
})

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydatabase'), 'python')

In [17]:
collections = db.list_collection_names()
collections

['python']

In [26]:
collection = db["python"]

In [27]:
collection.insert_one(data)

InsertOneResult(ObjectId('687be7e8efa723613f0019fa'), acknowledged=True)